In [1]:
# setup
import time, os, calendar, sys
import envkey
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
import seaborn as sns
from sqlalchemy.engine import create_engine
import math
import datetime

# allow importing modules from ../..
sys.path.insert(1, os.path.join(sys.path[0], '../..'))

register_matplotlib_converters()
sns.set(rc={'figure.figsize':(11, 4)})

ANALYTICS_MYSQL_ENDPOINT = os.getenv('ANALYTICS_MYSQL_ENDPOINT')
ANALYTICS_MYSQL_PASSWORD = os.getenv('ANALYTICS_MYSQL_PASSWORD')

if not (ANALYTICS_MYSQL_ENDPOINT and ANALYTICS_MYSQL_PASSWORD):
    raise Exception('ANALYTICS_MYSQL credentials not found')

def get_engine(game):
    return create_engine(f'mysql://analytics1:{ANALYTICS_MYSQL_PASSWORD}@{ANALYTICS_MYSQL_ENDPOINT}/{game}_upcache', connect_args={'connect_timeout': 10})

def get_auto_clicker_sql(min_time):
    return f"""
            select 'TR' as game, time AS day, user_id, 1.0*spec AS latency
            from tr_upcache.tr_metrics where time >= {min_time} and event_name = '3974_ui_latency' and (1.0*spec) < 0.4
            UNION ALL
            select 'MF' as game, time AS day, user_id, 1.0*spec AS latency
            from mf_upcache.mf_metrics where time >= {min_time} and event_name = '3974_ui_latency' and (1.0*spec) < 0.4
            UNION ALL
            select 'DV' as game, time AS day, user_id, 1.0*spec AS latency
            from dv_upcache.dv_metrics where time >= {min_time} and event_name = '3974_ui_latency' and (1.0*spec) < 0.4
            UNION ALL
            select 'MF2' as game, time AS day, user_id, 1.0*spec AS latency
            from mf2_upcache.mf2_metrics where time >= {min_time} and event_name = '3974_ui_latency' and (1.0*spec) < 0.4
            UNION ALL
            select 'BFM' as game, time AS day, user_id, 1.0*spec AS latency
            from bfm_upcache.bfm_metrics where time >= {min_time} and event_name = '3974_ui_latency' and (1.0*spec) < 0.4
            UNION ALL
            select 'FS' as game, time AS day, user_id, 1.0*spec AS latency
            from fs_upcache.fs_metrics where time >= {min_time} and event_name = '3974_ui_latency' and (1.0*spec) < 0.4
            order by latency asc
            """

def get_clock_tamper_sql(min_time):
    return f"""
            select 'BFM' as Game, user_id, COUNT(1) AS NumHits
            from bfm_upcache.bfm_metrics
            where time >= {min_time} and event_name = '3975_clock_race'
            and (1.0*spec > 1.45)
            group by user_id
            having NumHits >= 30
            UNION ALL
            select 'TR' as Game, user_id, COUNT(1) AS NumHits
            from tr_upcache.tr_metrics
            where time >= {min_time} and event_name = '3975_clock_race'
            and (1.0*spec > 1.45)
            group by user_id
            having NumHits >= 30
            UNION ALL
            select 'DV' as Game, user_id, COUNT(1) AS NumHits
            from dv_upcache.dv_metrics
            where time >= {min_time} and event_name = '3975_clock_race'
            and (1.0*spec > 1.45)
            group by user_id
            having NumHits >= 30
            UNION ALL
            select 'MF2' as Game, user_id, COUNT(1) AS NumHits
            from mf2_upcache.mf2_metrics
            where time >= {min_time} and event_name = '3975_clock_race'
            and (1.0*spec > 1.45)
            group by user_id
            having NumHits >= 30
            UNION ALL
            select 'FS' as Game, user_id, COUNT(1) AS NumHits
            from fs_upcache.fs_metrics
            where time >= {min_time} and event_name = '3975_clock_race'
            and (1.0*spec > 1.45)
            group by user_id
            having NumHits >= 30
            order by NumHits desc;
            """

def get_detailed_clock_tamper_sql(game,user_id,min_time):
    return f"""
            select time, 1.0*spec AS ratio
            from `{game}_upcache`.`{game}_metrics`
            where time >= {min_time} and event_name = '3975_clock_race' and user_id = {user_id}
            order by time desc;
            """

print(f"Dashboard updated %s" % time.strftime('%a, %d %b %Y at %H:%M:%S UTC', time.gmtime()))

Dashboard updated Sat, 29 Jan 2022 at 03:09:31 UTC


In [2]:
#  Battalion Recall Auto-Clickers (All Games)
min_time = time.time() - (14 * 86400)
min_time_str = datetime.date.fromtimestamp(time.time()) - datetime.timedelta(14)
updated_time = min_time_str.strftime('%a, %d %b %Y')
engine = get_engine('tr')
latency_sql = get_auto_clicker_sql(min_time)
latency_data = pd.read_sql(latency_sql, engine)
if len(latency_data) > 0:
    print('')
    print(f'\nBattalion Recall Auto-Clickers (All Games), showing incidents since {updated_time}')
    header_line = "{0:12} {1:10} {2:14} {3:14}".format('Date', 'User ID', 'Game', 'Latency')
    print(header_line)
    for i, day in enumerate(latency_data['day']):
        formatted_date = datetime.date.fromtimestamp(day).strftime('%Y-%m-%d')
        user_id = latency_data['user_id'][i]
        game_id = latency_data['game'][i]
        latency = latency_data['latency'][i]
        output_line = "{0:12} {1:10} {2:14} {3:14}".format(formatted_date, str(user_id), str(game_id),str(latency))
        print(output_line)
else:
    print(f'\nChecked Battalion Recall Auto-Clickers (All Games) since {updated_time}, but none found.')



Battalion Recall Auto-Clickers (All Games), showing incidents since Sat, 15 Jan 2022
Date         User ID    Game           Latency       
2022-01-20   4026249    TR             0.036         
2022-01-19   319034     BFM            0.041         
2022-01-20   4035782    TR             0.042         
2022-01-20   4035782    TR             0.045         
2022-01-15   319034     BFM            0.054         
2022-01-19   319034     BFM            0.055         
2022-01-20   4026249    TR             0.059         
2022-01-20   4026249    TR             0.061         
2022-01-20   4026249    TR             0.062         
2022-01-20   4026249    TR             0.063         
2022-01-20   4026249    TR             0.063         
2022-01-19   319034     BFM            0.064         
2022-01-20   4026249    TR             0.064         
2022-01-20   4026249    TR             0.065         
2022-01-20   4026249    TR             0.066         
2022-01-20   4026249    TR             0.067     

In [3]:
# Clock Tampering Detection (All Games)

#  Battalion Recall Auto-Clickers (All Games)
min_time = time.time() - (14 * 86400)
min_time_str = datetime.date.fromtimestamp(time.time()) - datetime.timedelta(14)
updated_time = min_time_str.strftime('%a, %d %b %Y')
engine = get_engine('tr')
latency_sql = get_clock_tamper_sql(min_time)
latency_data = pd.read_sql(latency_sql, engine)
detailed_clocks = {}
if len(latency_data) > 0:
    print('')
    print(f'Clock Tampering Detection (All Games), showing incidents since {updated_time}')
    header_line = "{0:4} {1:10} {2:5}".format('Game', 'User ID', 'Hits')
    print(header_line)
    for i, game_id in enumerate(latency_data['Game']):
        if game_id not in detailed_clocks:
            detailed_clocks[game_id] = []
        user_id = latency_data['user_id'][i]
        if user_id not in detailed_clocks[game_id]:
            detailed_clocks[game_id].append(user_id)
        hits = latency_data['NumHits'][i]
        output_line = "{0:12} {1:10} {2:5}".format(game_id, str(user_id), str(hits))
        print(output_line)
else:
    print(f'Clock Tampering Detection (All Games) since {updated_time}, but none found.')


Clock Tampering Detection (All Games), showing incidents since Sat, 15 Jan 2022
Game User ID    Hits 
TR           889913     55   
DV           185171     40   
TR           4154820    30   


In [6]:
if len(detailed_clocks.keys()) > 0:
    print('* Players listed above might be false positives. To run a detailed check, look at the "ratio" numbers on each page.')
    print('If the numbers seem random, then the player is innocent.')
    print('If you see many "ratios" clustered very close to a number higher than 1.0 (e..g, 1.500, 1.499, 1.501, 1.502, ...), then the player is definitely tampering with their clock.)')
    for game in detailed_clocks:
        user_ids = detailed_clocks[game]
        min_time = time.time() - (14 * 86400)
        engine = get_engine(game.lower())
        for user_id in user_ids:
            latency_sql = get_detailed_clock_tamper_sql(game.lower(),user_id,min_time)
            latency_data = pd.read_sql(latency_sql, engine)
            if len(latency_data) > 0:
                print(f'Clock Tampering Details for user {user_id} ({game})')
                header_line = "{0:12} {1:10}".format('Time', 'Ratio')
                print(header_line)
                for i, time_val in enumerate(latency_data['time']):
                    ratio = latency_data['ratio'][i]
                    output_line = "{0:12} {1:10}".format(str(time_val), str(ratio))
                    print(output_line)

* Players listed above might be false positives. To run a detailed check, look at the "ratio" numbers on each page.
If the numbers seem random, then the player is innocent.
If you see many "ratios" clustered very close to a number higher than 1.0 (e..g, 1.500, 1.499, 1.501, 1.502, ...), then the player is definitely tampering with their clock.)
Clock Tampering Details for user 889913 (TR)
Time         Ratio     
1643290517   1.0       
1643290426   1.0003    
1643290302   7157.6454 
1643290129   0.2738    
1643290032   0.9995    
1643289602   2.444     
1643289418   0.755     
1643289332   1.0001    
1643289275   1.008     
1643288631   1.0044    
1643288550   1.0666    
1643288217   0.2283    
1643288001   0.6222    
1643287888   1.3517    
1643287852   7.1709    
1643287852   45302.1446
1643287671   1.4707    
1643287286   1.0081    
1643287183   1.0018    
1643287126   0.999     
1643287024   1.0       
1643286817   1.0302    
1643286799   0.9961    
1643286593   0.997     
16432864